In [2]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 5.8 MB/s 


In [3]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('ggplot')

plt.rc('figure', autolayout=True)

tf.__version__

'2.8.2'

In [7]:
train_dataset = h5py.File('https://github.com/ThePiranhaGuy/hand_sign_keras/blob/main/datasets/train_signs.h5?raw=true', "r")
test_dataset = h5py.File('https://github.com/ThePiranhaGuy/hand_sign_keras/blob/main/datasets/test_signs.h5?raw=true', "r")

OSError: ignored

In [ ]:
x_train = np.array(train_dataset['train_set_x'])
y_train = np.array(train_dataset['train_set_y'])
x_test = np.array(test_dataset['test_set_x'])
y_test = np.array(test_dataset['test_set_y'])

In [ ]:
print(type(x_train))
print(x_train.shape)


In [ ]:
unique_labels = np.unique(y_train)
print(unique_labels,len(unique_labels))

In [ ]:

plt.figure(figsize=(10, 10))
for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i])
    plt.title(y_train[i])
    plt.axis("off")

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(128, activation = tf.nn.relu),
                                    tf.keras.layers.Dense(6, activation = tf.nn.softmax)])


In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics =['accuracy'])


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")

In [ ]:
model.evaluate(x_test, y_test)


4/4 [==============================] - 0s 6ms/step - loss: 0.8531 - accuracy: 0.6333


[0.8530899882316589, 0.6333333253860474]

Tuning

In [ ]:
def evaluate_model(model, X_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

In [ ]:

# Evaluate model on test set and add results to dataframe
results = evaluate_model(model, x_test, y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

4/4 [==============================] - 0s 5ms/step - loss: 0.8531 - accuracy: 0.6333


,loss,accuracy
Baseline,0.85309,0.633333


In [ ]:
def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 4
    # Number of Units: 64 - 512 with stepsize of 64
    for i in range(1, hp.Int("num_layers", 2, 5)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=64, max_value=512, step=64),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=6, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [ ]:
# Instantiate the tuner
import keras_tuner as kt
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir",
                     project_name="kt_hyperband",)
# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}


In [ ]:
stop_early = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

tuner.search(x_train, y_train, epochs=100, validation_split=0.2, callbacks=[stop_early], verbose=2)

Trial 299 Complete [00h 00m 11s]
val_accuracy: 0.21296297013759613

Best val_accuracy So Far: 0.6851851940155029
Total elapsed time: 00h 43m 34s

Search: Running Trial #300

Value             |Best Value So Far |Hyperparameter
4                 |3                 |num_layers
320               |448               |units_1
0.1               |0                 |dropout_1
0.0001            |0.0001            |learning_rate
320               |256               |units_2
0.1               |0.1               |dropout_2
128               |320               |units_3
0                 |0.1               |dropout_3
256               |384               |units_4
0.2               |0.2               |dropout_4
20                |20                |tuner/epochs
0                 |0                 |tuner/initial_epoch
0                 |0                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/20
27/27 - 2s - loss: 1.9157 - accuracy: 0.1817 - val_loss: 1.7511 - val_acc